# testing

> Standardized test page creation for Jupyter notebooks with FastHTML

In [ ]:
#| default_exp core.testing

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fasthtml.common import *
from fasthtml.jupyter import JupyUvi, HTMX
from typing import Optional, Union, List, Callable
from pathlib import Path

# Import our modules
from cjm_fasthtml_daisyui.core.resources import build_headers, get_daisyui_headers
from cjm_fasthtml_daisyui.core.themes import DaisyUITheme, get_theme_value
from cjm_fasthtml_daisyui.core.colors import DaisyUIColor, enable_semantic_gradients

# Import TailwindBuilder with recommended pattern
from cjm_tailwind_utils.all import TailwindBuilder

## Test App Creation

A standardized way to create test apps in Jupyter notebooks:

In [ ]:
#| export
def create_test_app(
    theme: Union[DaisyUITheme, str] = DaisyUITheme.LIGHT,  # Default theme
    include_theme_selector: bool = True,  # Include theme selector in app
    custom_css: Optional[List[Union[str, Link]]] = None,  # Additional CSS
    custom_js: Optional[List[Union[str, Script]]] = None,  # Additional JS
    custom_theme_css: Optional[str] = None,  # Custom theme CSS as string
    custom_theme_paths: Optional[List[Union[str, Path]]] = None,  # List of paths to custom theme CSS files
    custom_theme_names: Optional[List[str]] = None,  # Names of custom themes to include in selector
    debug: bool = True  # Enable debug mode
) -> tuple: # Tuple containing (app, rt) - FastHTML app instance and route decorator
    """
    Create a standardized test app for Jupyter notebooks with daisyUI and Tailwind.
    """
    # Get theme value - allow custom themes when custom CSS, paths or names are provided
    theme_value = get_theme_value(theme, allow_custom=bool(custom_theme_paths or custom_theme_css or custom_theme_names))
    
    # Build headers
    headers = build_headers(
        include_themes=True,
        custom_css=custom_css,
        custom_js=custom_js,
        custom_theme_css=custom_theme_css,
        custom_theme_paths=custom_theme_paths
    )
    
    # Create app with daisyUI configuration
    app, rt = fast_app(
        pico=False,  # Disable pico since we're using daisyUI
        hdrs=headers,
        htmlkw={'data-theme': theme_value},
        debug=debug
    )
    
    # Add theme selector route if requested
    if include_theme_selector:
        @rt('/theme-selector')
        def theme_selector(
        ) -> Div: # Theme selector dropdown component
            "Route handler that returns the theme selector dropdown component"
            return create_theme_selector(custom_themes=custom_theme_names)
    
    return app, rt

## Theme Selector Component

A reusable theme selector for testing:

In [ ]:
#| export
def create_theme_selector(
    custom_themes: Optional[List[str]] = None  # Optional list of custom theme names to include
) -> Div:  # Div containing theme selector dropdown with theme-change script
    "Create a daisyUI theme selector dropdown component. Uses theme-change library to persist theme selection in localStorage."
    # Build classes using TailwindBuilder for better readability and type safety
    dropdown_classes = "dropdown dropdown-end"
    button_classes = "btn btn-ghost btn-circle"
    
    # Use TailwindBuilder for layout/spacing utilities with method chaining
    # Note: rounded-box is a daisyUI utility, kept as raw string
    dropdown_content_classes = (
        TailwindBuilder()
        .z(10)
        .p(2)
        .shadow("2xl")
        .w(52)
        .bg_semantic(DaisyUIColor.BASE_100)
        .util("dropdown-content", "rounded-box")
        .build()
    )
    
    menu_classes = "menu menu-xs menu-dropdown-toggle"
    
    # Build theme options
    theme_options = []
    
    # Add custom themes first if provided
    if custom_themes:
        for theme_name in custom_themes:
            theme_options.append(Li(
                Input(
                    type="radio",
                    name="theme-dropdown",
                    cls="theme-controller btn btn-sm btn-block btn-ghost justify-start",
                    aria_label=f"{theme_name.title()}",
                    value=theme_name
                )
            ))
            
        # Add a divider between custom and built-in themes
        if custom_themes:
            theme_options.append(Li(Div(cls="divider my-0")))
    
    # Add built-in themes
    for theme in DaisyUITheme:
        theme_options.append(Li(
            Input(
                type="radio",
                name="theme-dropdown",
                cls="theme-controller btn btn-sm btn-block btn-ghost justify-start",
                aria_label=theme.value.title(),
                value=theme.value
            )
        ))
    
    return Div(
        # Theme selector dropdown
        Div(
            Label(
                "🎨",
                tabindex="0",
                cls=button_classes
            ),
            Ul(
                *theme_options,
                tabindex="0",
                cls=dropdown_content_classes + " " + menu_classes
            ),
            cls=dropdown_classes
        ),
        # Theme change script
        Script(src="https://cdn.jsdelivr.net/npm/theme-change@2.0.2/index.min.js"),
        Script("document.addEventListener('DOMContentLoaded', () => themeChange());")
    )

## Test Page Wrapper

A wrapper for creating consistent test pages:

In [ ]:
#| export
def create_test_page(
    title: str,  # Page title
    *content,  # Page content elements
    include_theme_selector: bool = True,  # Include theme selector
    container: bool = True,  # Wrap in container
    custom_theme_names: Optional[List[str]] = None  # Custom themes for selector
) -> Div:  # Div containing complete page layout with navbar and content
    """
    Create a standardized test page layout with optional theme selector.
    """
    # Build navbar with proper structure using TailwindBuilder
    navbar = Div(
        # Container for navbar content with flex layout
        Div(
            # Left section (empty for now)
            Div(cls=TailwindBuilder().flex(1).build()),
            
            # Center section with title
            Div(
                H1(title, cls=TailwindBuilder().text("2xl").font("bold").build()),
                cls=TailwindBuilder().flex(None).build()
            ),
            
            # Right section with theme selector
            Div(
                create_theme_selector(custom_themes=custom_theme_names) if include_theme_selector else "",
                cls=TailwindBuilder().flex().justify("end").flex(1).build()
            ),
            cls=(
                TailwindBuilder()
                .container()
                .m("auto", "x")
                .flex()
                .items("center")
                .build()
            )
        ),
        cls=TailwindBuilder().util("navbar").bg_semantic(DaisyUIColor.BASE_100).shadow("lg").build()
    )
    
    # Build main content classes with TailwindBuilder
    main_builder = TailwindBuilder()
    if container:
        main_builder = main_builder.container().m("auto", "x")
    main_classes = main_builder.p(4, "x").p(8, "y").build()
    
    return Div(
        navbar,
        Main(
            *content,
            cls=main_classes
        )
    )

## Jupyter Notebook Utilities

Helper functions for working with FastHTML in Jupyter:

In [ ]:
#| export
def start_test_server(
    app: FastHTML,    # FastHTML app instance created by create_test_app or fast_app
    port: int = 8000,  # Port
) -> JupyUvi:  # JupyUvi server instance for Jupyter notebook testing
    """
    Start a test server and return the JupyUvi instance.
    
    Usage:
        server = start_test_server(app)
        HTMX()  # Display the app
        
        # Later, in another cell:
        server.stop()
    """
    return JupyUvi(app, port=port)

## Example Usage

Here's how to use the testing utilities in a notebook:

In [ ]:
#| eval: false
# Create a test app with default settings
app, rt = create_test_app(theme=DaisyUITheme.LIGHT)

# Define a test route
@rt
def index():
    return create_test_page(
        "Component Test Page",
        Div(
            H2("Test Components", cls=TailwindBuilder().text("2xl").m(4, side='b').build()),
            P("This is a test page for daisyUI components.", cls=TailwindBuilder().text_semantic(DaisyUIColor.BASE_CONTENT).build()),
            Div(
                Button("Primary", cls="btn btn-primary"),
                Button("Secondary", cls="btn btn-secondary"),
                Button("Accent", cls="btn btn-accent"),
                cls=TailwindBuilder().space(1, "x").build()
            )
        )
    )

# Start the server
server = start_test_server(app)
HTMX()

In [ ]:
#| eval: false
# Stop the server when done
server.stop()

## Loading Custom Themes from CSS Files

You can also load custom themes from CSS files using Path objects:

In [ ]:
#| eval: false
# First, let's create a second custom theme and save it to a CSS file
from cjm_fasthtml_daisyui.core.themes import ThemeConfig, save_theme_css
from nbdev.config import get_config

# Get project directory
cfg = get_config()
project_dir = cfg.config_path
css_dir = project_dir / "css"
css_dir.mkdir(exist_ok=True, parents=True)

# Create a theme
custom_theme: ThemeConfig = {
    "name": "netwatch_cyberpunk",
    "default": False,
    "prefersdark": True,
    "color_scheme": "dark",
    "colors": {
        "base_100": "oklch(6.72% 0.000 0)",
        "base_200": "oklch(12% 0.005 240)",
        "base_300": "oklch(8% 0.01 235)",
        "base_content": "oklch(79.82% 0.136 184.06)",
        "primary": "oklch(61.39% 0.244 12.03)",
        "primary_content": "oklch(6.72% 0.000 0)",
        "secondary": "oklch(46.40% 0.184 10.98)",
        "secondary_content": "oklch(79.82% 0.136 184.06)",
        "accent": "oklch(25.77% 0.075 12.95)",
        "accent_content": "oklch(79.82% 0.136 184.06)",
        "neutral": "oklch(20% 0.01 240)",
        "neutral_content": "oklch(79.82% 0.136 184.06)",
        "info": "oklch(79.82% 0.136 184.06)",
        "info_content": "oklch(6.72% 0.000 0)",
        "success": "oklch(62% 0.20 140)",
        "success_content": "oklch(6.72% 0.000 0)",
        "warning": "oklch(75% 0.25 60)",
        "warning_content": "oklch(6.72% 0.000 0)",
        "error": "oklch(61.39% 0.244 12.03)",
        "error_content": "oklch(6.72% 0.000 0)"
    },
    "radius_selector": "0rem",
    "radius_field": "0rem",
    "radius_box": "0rem",
    "size_selector": "0.125rem",
    "size_field": "0.125rem",
    "border": "1px",
    "depth": 0,
    "noise": 0
}

# Save themes to CSS files
save_theme_css(custom_theme, css_dir / f"{custom_theme['name']}.css")

print(f"Created theme files in: {css_dir}")

Created theme files in: /mnt/SN850X_8TB_EXT4/Projects/GitHub/cj-mills/cjm-fasthtml-daisyui/css


In [ ]:
#| eval: false
# Now create an app that loads custom themes from CSS files
custom_theme_paths=list(css_dir.glob("*.css"))

custom_theme_names=[theme.stem for theme in custom_theme_paths]
custom_theme_names.sort()

app, rt = create_test_app(
    theme=custom_theme_names[0],  # Use one of our custom themes as default
    custom_theme_paths=custom_theme_paths,
    custom_theme_names=custom_theme_names,  # Add both custom themes to selector
    include_theme_selector=True
)

# Create a test page showcasing both custom themes
@rt
def index():
    return create_test_page(
        "Custom Themes from CSS Files",
        Div(
            # Info card
            Div(
                H2("Loading Themes from Files", cls="card-title"),
                P("This example demonstrates loading custom themes from CSS files using Path objects."),
                P("The themes are loaded as Style elements in the page header, making them available throughout the app."),
                P("Switch between 'test_theme' and 'dark_custom' themes using the selector above!"),
                cls=(
                    TailwindBuilder()
                    .util("card")
                    .bg_semantic(DaisyUIColor.BASE_200)
                    .shadow("xl")
                    .p(6)
                    .m(8, side='b')
                    .build()
                )
            ),
            
            # Components showcase
            H3("Theme Components", cls=TailwindBuilder().text("xl").font("bold").m(4, side='b').build()),
            
            # Buttons
            Div(
                H4("Buttons", cls=TailwindBuilder().font("semibold").m(2, side='b').build()),
                Div(
                    Button("Primary", cls="btn btn-primary"),
                    Button("Secondary", cls="btn btn-secondary"),
                    Button("Accent", cls="btn btn-accent"),
                    Button("Neutral", cls="btn btn-neutral"),
                    cls=TailwindBuilder().space(2, "x").build()
                ),
                cls=TailwindBuilder().m(6, side='b').build()
            ),
            
            # Cards
            Div(
                H4("Cards", cls=TailwindBuilder().font("semibold").m(2, side='b').build()),
                Div(
                    Div(
                        H3("Base 100 Card", cls="card-title"),
                        P("This card uses the base-100 background color"),
                        cls=(
                            TailwindBuilder()
                            .util("card")
                            .bg_semantic(DaisyUIColor.BASE_100)
                            .shadow()
                            .p(4)
                            .build()
                        )
                    ),
                    Div(
                        H3("Base 200 Card", cls="card-title"),
                        P("This card uses the base-200 background color"),
                        cls=(
                            TailwindBuilder()
                            .util("card")
                            .bg_semantic(DaisyUIColor.BASE_200)
                            .shadow()
                            .p(4)
                            .build()
                        )
                    ),
                    Div(
                        H3("Base 300 Card", cls="card-title"),
                        P("This card uses the base-300 background color"),
                        cls=(
                            TailwindBuilder()
                            .util("card")
                            .bg_semantic(DaisyUIColor.BASE_300)
                            .shadow()
                            .p(4)
                            .build()
                        )
                    ),
                    cls=TailwindBuilder().grid(cols=3).gap(4).build()
                ),
                cls=TailwindBuilder().m(6, side='b').build()
            ),
            
            # Alerts
            Div(
                H4("Alerts", cls=TailwindBuilder().font("semibold").m(2, side='b').build()),
                Div(
                    Div("Info alert message", cls="alert alert-info"),
                    Div("Success alert message", cls="alert alert-success"),
                    Div("Warning alert message", cls="alert alert-warning"),
                    Div("Error alert message", cls="alert alert-error"),
                    cls=TailwindBuilder().space(2, "y").build()
                )
            )
        ),
        custom_theme_names=custom_theme_names
    )

# Start the server
server = start_test_server(app, port=8000)
HTMX()

In [ ]:
#| eval: false
# Stop the server when done
server.stop()

## Custom JavaScript Support

You can include custom JavaScript files and inline scripts in your test app:

In [ ]:
#| eval: false
# Create a test app with custom JavaScript
from cjm_fasthtml_daisyui.core.resources import create_js_script

# Create inline custom JavaScript that adds interactivity
custom_js_inline = Script("""
// Custom JavaScript to add click counter functionality
document.addEventListener('DOMContentLoaded', function() {
    let clickCount = 0;
    const counterElement = document.getElementById('click-counter');
    const resetButton = document.getElementById('reset-counter');
    
    // Add click listener to all buttons with 'count-click' class
    document.querySelectorAll('.count-click').forEach(button => {
        button.addEventListener('click', function() {
            clickCount++;
            if (counterElement) {
                counterElement.textContent = clickCount;
                
                // Add visual feedback
                counterElement.classList.add('text-primary', 'font-bold');
                setTimeout(() => {
                    counterElement.classList.remove('text-primary');
                }, 300);
            }
        });
    });
    
    // Reset counter functionality
    if (resetButton) {
        resetButton.addEventListener('click', function() {
            clickCount = 0;
            if (counterElement) {
                counterElement.textContent = clickCount;
            }
        });
    }
    
    // Log to console to verify script is loaded
    console.log('Custom JavaScript loaded successfully!');
});
""")

# Create a custom external JavaScript reference (example using a CDN library)
# In this case, we'll use confetti for visual effects
confetti_js = create_js_script(
    src="https://cdn.jsdelivr.net/npm/canvas-confetti@1.9.2/dist/confetti.browser.min.js",
    defer=True
)

# Create app with custom JavaScript
app, rt = create_test_app(
    theme=DaisyUITheme.CUPCAKE,
    custom_js=[confetti_js, custom_js_inline],
    include_theme_selector=True
)

# Create a test page that uses the custom JavaScript
@rt
def index():
    return create_test_page(
        "Custom JavaScript Test",
        Div(
            # Info card
            Div(
                H2("JavaScript Integration", cls="card-title"),
                P("This example demonstrates custom JavaScript support in test apps."),
                P("Click the buttons below to see the click counter in action!"),
                Div(
                    "Total clicks: ",
                    Span("0", id="click-counter", cls="text-2xl font-bold"),
                    cls=TailwindBuilder().text("lg").m(4, side='t').build()
                ),
                cls=(
                    TailwindBuilder()
                    .util("card")
                    .bg_semantic(DaisyUIColor.BASE_200)
                    .shadow("xl")
                    .p(6)
                    .m(8, side='b')
                    .build()
                )
            ),
            
            # Interactive buttons
            Div(
                H3("Click Counter Demo", cls=TailwindBuilder().text("xl").font("bold").m(4, side='b').build()),
                Div(
                    Button("Click Me!", cls="btn btn-primary count-click"),
                    Button("Me Too!", cls="btn btn-secondary count-click"),
                    Button("And Me!", cls="btn btn-accent count-click"),
                    Button("Reset Counter", id="reset-counter", cls="btn btn-neutral"),
                    cls=TailwindBuilder().space(2, "x").build()
                ),
                cls=TailwindBuilder().m(6, side='b').build()
            ),
            
            # Confetti button
            Div(
                H3("Visual Effects Demo", cls=TailwindBuilder().text("xl").font("bold").m(4, side='b').build()),
                P("Using an external JavaScript library (canvas-confetti):"),
                Button(
                    "🎉 Celebrate!",
                    cls="btn btn-lg btn-primary",
                    onclick="confetti({particleCount: 100, spread: 70, origin: { y: 0.6 }})"
                ),
                cls=TailwindBuilder().m(6, side='b').build()
            ),
            
            # Console message
            Div(
                P("💡 Open your browser's developer console to see the custom JavaScript log message."),
                cls=(
                    TailwindBuilder()
                    .util("alert", "alert-info")
                    .m(4, side='t')
                    .build()
                )
            )
        )
    )

# Start the server
server = start_test_server(app, port=8000)
HTMX()

In [ ]:
#| eval: false
# Stop the server when done
server.stop()

## Debug Mode and String Theme Names

You can disable debug mode and use theme names as strings instead of enum values:

In [ ]:
#| eval: false
# Create app with debug=False and string theme name
app, rt = create_test_app(
    theme="cyberpunk",  # Using string instead of DaisyUITheme.CYBERPUNK
    debug=False,  # Disable debug mode for production-like behavior
    include_theme_selector=True
)

# Create a simple test page to verify the configuration
@rt
def index():
    return create_test_page(
        "Production Mode Test",
        Div(
            # Info card
            Div(
                H2("Testing Production Settings", cls="card-title"),
                P("This app demonstrates:"),
                Ul(
                    Li("✓ Debug mode disabled (debug=False)"),
                    Li("✓ Theme specified as string ('cyberpunk') instead of enum"),
                    Li("✓ No debug error pages or hot reloading"),
                    cls=TailwindBuilder().util("list-disc").m(6, side='l').space(1, "y").build()
                ),
                cls=(
                    TailwindBuilder()
                    .util("card")
                    .bg_semantic(DaisyUIColor.BASE_200)
                    .shadow("xl")
                    .p(6)
                    .m(8, side='b')
                    .build()
                )
            ),
            
            # Visual test of cyberpunk theme
            Div(
                H3("Cyberpunk Theme Elements", cls=TailwindBuilder().text("xl").font("bold").m(4, side='b').build()),
                P("The cyberpunk theme features sharp edges and high contrast:"),
                Div(
                    Button("Primary", cls="btn btn-primary"),
                    Button("Secondary", cls="btn btn-secondary"),
                    Button("Accent", cls="btn btn-accent"),
                    Input(type="text", placeholder="Cyberpunk input", cls="input input-bordered"),
                    cls=TailwindBuilder().space(2, "x").items("center").flex().build()
                )
            )
        )
    )

# Start the server
server = start_test_server(app, port=8003)
HTMX(port=8003)

In [ ]:
#| eval: false
# Stop the server when done
server.stop()

## Full-Width Layout (No Container)

Use `container=False` to create full-width layouts without the container constraint:

In [ ]:
#| eval: false
# Create app with a full-width layout example
app, rt = create_test_app(theme=DaisyUITheme.BUSINESS)

# Create routes to demonstrate both container and non-container layouts
@rt
def index():
    return create_test_page(
        "Full-Width Layout Demo",
        Div(
            # Hero section that spans full width
            Div(
                enable_semantic_gradients(),  # Enable gradient support for semantic colors
                Div(
                    H2("Full-Width Hero Section", cls="text-5xl font-bold text-white"),
                    P("This layout uses container=False for edge-to-edge content", cls="text-xl text-white/80 mt-4"),
                    cls=TailwindBuilder().text("center").p(20, side='y').p(4, side='x').build()
                ),
                cls=(
                    TailwindBuilder()
                    .util("bg-gradient-to-r", "from-primary", "to-secondary")
                    .m(4, side='b')
                    .build()
                )
            ),
            
            # Content grid that spans full width
            Div(
                # Stats across full width
                Div(
                    Div("100%", H3("Width", cls="text-sm opacity-60"), cls=TailwindBuilder().util("stat-value").text_semantic(DaisyUIColor.PRIMARY).build()),
                    cls="stat place-items-center"
                ),
                Div(
                    Div("Edge", H3("To Edge", cls="text-sm opacity-60"), cls=TailwindBuilder().util("stat-value").text_semantic(DaisyUIColor.SECONDARY).build()),
                    cls="stat place-items-center"
                ),
                Div(
                    Div("No", H3("Container", cls="text-sm opacity-60"), cls=TailwindBuilder().util("stat-value").text_semantic(DaisyUIColor.ACCENT).build()),
                    cls="stat place-items-center"
                ),
                cls=(
                    TailwindBuilder()
                    .util("stats", "shadow")
                    .bg_semantic(DaisyUIColor.BASE_200)
                    .w("full")
                    .build()
                )
            ),
            
            # Feature cards in a full-width grid
            Div(
                H3("Full-Width Grid Layout", cls=TailwindBuilder().text("2xl").font("bold").m(8, side='b').p(4, side='x').build()),
                Div(
                    *[Div(
                        H4(f"Feature {i+1}", cls="card-title"),
                        P("This card is part of a full-width responsive grid layout"),
                        cls=(
                            TailwindBuilder()
                            .util("card", "shadow-xl")
                            .bg_semantic(DaisyUIColor.BASE_100)
                            .p(6)
                            .build()
                        )
                    ) for i in range(6)],
                    cls=(
                        TailwindBuilder()
                        .grid()
                        .grid(cols=1)
                        .md().grid(cols=2)
                        .lg().grid(cols=3)
                        .gap(4)
                        .p(4, side='x')
                        .build()
                    )
                )
            )
        ),
        container=False,  # Disable container constraint
        include_theme_selector=True
    )

# Comparison route with container=True (default)
@rt('/with-container')
def with_container():
    return create_test_page(
        "Container Layout Demo",
        Div(
            Div(
                H2("Container-Constrained Layout", cls="text-3xl font-bold mb-4"),
                P("This page uses the default container=True setting, which centers content and limits max width."),
                Div(
                    "The container utility applies responsive max-widths and centers the content horizontally.",
                    cls="alert alert-info mt-4"
                ),
                cls="mb-8"
            ),
            
            # Same stats component but within container
            Div(
                Div("Limited", H3("Width", cls="text-sm opacity-60"), cls=TailwindBuilder().util("stat-value").text_semantic(DaisyUIColor.PRIMARY).build()),
                cls="stat place-items-center"
            ),
            Div(
                Div("Centered", H3("Content", cls="text-sm opacity-60"), cls=TailwindBuilder().util("stat-value").text_semantic(DaisyUIColor.SECONDARY).build()),
                cls="stat place-items-center"
            ),
            Div(
                Div("Yes", H3("Container", cls="text-sm opacity-60"), cls=TailwindBuilder().util("stat-value").text_semantic(DaisyUIColor.ACCENT).build()),
                cls="stat place-items-center"
            ),
            cls=TailwindBuilder().util("stats", "shadow").bg_semantic(DaisyUIColor.BASE_200).build()
        ),
        container=True  # This is the default, shown explicitly for clarity
    )

# Start the server
server = start_test_server(app, port=8004)
HTMX(port=8004)

In [ ]:
#| eval: false
# Stop the server when done
server.stop()

In [ ]:
TailwindBuilder().util("bg-gradient-to-r", f"from-{DaisyUIColor.PRIMARY.value}", f"to-{DaisyUIColor.SECONDARY.value}").m(4, side='b').build()

'bg-gradient-to-r from-primary mb-4 to-secondary'

In [ ]:
DaisyUIColor.PRIMARY.value

'primary'

## Export

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()